In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow==2.10.0
!pip install pyserial

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 33.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorbo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import output
from IPython.display import display, Javascript
from PIL import Image
import io
import base64
import datetime
import time
from keras.models import load_model
import cv2
import numpy as np
import serial

In [ ]:
# JavaScript function to start the webcam and capture the video feed
def start_camera():
    display(Javascript("""
        async function startCamera() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            video.play();
            document.body.appendChild(video);

            // Create a canvas element to capture frames
            const canvas = document.createElement('canvas');
            canvas.width = 640;
            canvas.height = 480;
            const context = canvas.getContext('2d');
            document.body.appendChild(canvas);

            // Capture the first frame immediately
            function captureFrame() {
                context.drawImage(video, 0, 0, 640, 480);
                canvas.toBlob(function(blob) {
                    const reader = new FileReader();
                    reader.onloadend = function() {
                        const base64Image = reader.result.split(',')[1];
                        google.colab.kernel.invokeFunction('notebook.capture_frame', [base64Image], {});
                    };
                    reader.readAsDataURL(blob);
                }, 'image/png');
            }

            // Capture the image immediately
            captureFrame();
        }
        startCamera();
    """))

# Python function to process and save the captured frame
def capture_frame(base64_image):
    global image
    # Decode the base64 image
    img_data = np.frombuffer(base64.b64decode(base64_image), dtype=np.uint8)
    img = cv2.imdecode(img_data, cv2.IMREAD_COLOR)

    # Convert the image to RGB (from BGR) for display and saving
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)

    # Save the image with a timestamp as the filename
    filename = f"captured_frame_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    pil_img.save(filename)
    print(f"Image saved as {filename}")

    # Update the shared variable and signal the event
    image = img_rgb


  IMAGE CLASSIFICATION

In [ ]:
image =  None

# Register the Python function to be called from JavaScript
output.register_callback('notebook.capture_frame', capture_frame)

# Start the webcam in Google Colab
start_camera()

<IPython.core.display.Javascript object>

Image saved as captured_frame_20241112_200558.png


In [ ]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/drive/MyDrive/Colab Notebooks/EcoSense Bin/keras_model.h5", compile=True)

# Load the labels
class_names = ['1','2','3']

if image is None:
    time.sleep(5)  # Wait for 5 seconds

# Resize the image to 224x224 using OpenCV
image = cv2.resize(image, (224, 224))

# Make the image a numpy array and reshape it to the models input shape.
image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

# Normalize the image array
image = (image / 127.5) - 1

# Predicts the model
prediction = model.predict(image)
index = np.argmax(prediction)
predicted_class = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", predicted_class, end=", ")
print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

1/1 [==============================] - 1s 720ms/step
Class: 2, Confidence Score: 57 %


ARDUINO CONNECTION

In [ ]:
port = 'COM3'
baud_rate = 9600

# Connect to Arduino
try:
    arduino = serial.Serial(port, baud_rate)
    # Send data to Arduino as string
    arduino.write(str(predicted_class).encode())
    print(f"Data sent to {port}")

    # Listen to the keyboard for presses.
    cv2.waitKey(1)

except serial.SerialException as e:
    print(f"Failed to connect to {port}: {e}")

cv2.destroyAllWindows()

Failed to connect to COM3: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'
